In [7]:

file_path = r"knowledgeBase\pdfParsed_output\TOC_hierarchy_result_20260112_112248.md"
with open(file_path, 'r', encoding='utf-8') as file:
    raw_data = file.read()
print(f"数据加载成功，字符数：{len(raw_data)}")

数据加载成功，字符数：20882


1. 数据目录结构还原

In [ ]:
import re

# [Cell 2] 目录结构层级清洗
def clean_structure_headers(text):
    lines = text.split('\n')
    new_lines = []
    
    # 定义层级正则匹配规则
    # 匹配 "第一章" -> 一级标题
    p_h1 = re.compile(r'^\s*#*\s*(第[一二三四五六七八九十]+章.*)')
    # 匹配 "一、" -> 二级标题
    p_h2 = re.compile(r'^\s*#*\s*([一二三四五六七八九十]+、.*)')
    # 匹配 "(一)" 或 "（一）" -> 三级标题
    p_h3 = re.compile(r'^\s*#*\s*([(（][一二三四五六七八九十]+[)）].*)')
    
    for line in lines:
        stripped = line.strip()
        # 强制重写层级
        if p_h1.match(stripped):
            new_line = "# " + p_h1.match(stripped).group(1)
        elif p_h2.match(stripped):
            new_line = "## " + p_h2.match(stripped).group(1)
        elif p_h3.match(stripped):
            new_line = "### " + p_h3.match(stripped).group(1)
        else:
            new_line = line 
        new_lines.append(new_line)
    
    return '\n'.join(new_lines)

# 执行处理
step1_text = clean_structure_headers(raw_data)

print("--- 目录结构清洗后预览 (前10行) ---")
print('\n'.join(step1_text.split('\n')[:10]))

In [15]:
import re

def final_deep_clean(text):
    # 1. 预处理 HTML 标签
    text = text.replace('<br>', '\n').replace('<b>', '').replace('</b>', '')
    
    lines = text.split('\n')
    cleaned_lines = []
    seen_content = set() 

    for line in lines:
        # === A. 核心拦截：彻底删除表格分割线 ===
        # 匹配只包含 | - : 和 空格 的行 (Markdown表格线)
        if re.match(r'^[\|\s\-:]+$', line.strip()):
            continue
            
        # === B. 基础清理 ===
        # 去掉表格竖线 | 和 连续的填充点 ·
        line = line.replace('|', ' ')
        line = re.sub(r'[·\.]{2,}', ' ', line) # 匹配2个以上的连续点
        line = line.strip()

        # === C. 暴力去重 (处理滚雪球式叠加) ===
        # 如果一行里包含多个层级标志，只取最后一个
        parts = re.split(r'(?=第[一二三四五六七八九十]+章|[一二三四五六七八九十]+、|[(（][一二三四五六七八九十]+[)）]|\d+\.)', line)
        if len(parts) > 1:
            line = parts[-1].strip()

        # === D. 剔除杂质 ===
        # 1. 移除行末页码
        line = re.sub(r'\s*\d+$', '', line)
        # 2. 移除孤立的无意义字符 (如 OCR 误读的 I, l, _ )
        line = re.sub(r'^[Il_]\s*$', '', line)
        
        if not line or len(line) < 2:
            continue

        # === E. 重新标准化 Markdown 标题 ===
        if re.search(r'第[一二三四五六七八九十]+章', line):
            line = "# " + re.sub(r'^#+\s*', '', line)
        elif re.search(r'[一二三四五六七八九十]+、', line):
            line = "## " + re.sub(r'^#+\s*', '', line)
        elif re.search(r'[(（][一二三四五六七八九十]+[)）]', line):
            line = "### " + re.sub(r'^#+\s*', '', line)
        elif re.match(r'\d+\.', line):
            line = "#### " + re.sub(r'^#+\s*', '', line)

        # 行级去重
        if line not in seen_content:
            cleaned_lines.append(line)
            seen_content.add(line)

    return '\n'.join(cleaned_lines)

# 执行处理
step2_text = final_deep_clean(raw_data)
print(step2_text)

# 目 录
自 序
说 明
修订说明
# 第一章 总 论
## 一、《营造法式》的性质与特点
### (一)"营造法式"是一种建筑工程预算定额
### (二)李诫《营造法式》的编写体例
### (三)编写工作紧密结合实际
## 二、《营造法式》与江南建筑的关系
## 三、《营造法式》的内容取舍
## 四、宋代官式建筑分类
## 五、研究方法的讨论
# 第一章注释
# 第二章 木构架
## 一、宋代官式建筑木构架的基本类型
### (一)柱梁作
### (二)殿阁式木构架
#### 1. 柱框层
#### 2. 铺作层
#### 3. 屋盖层
### (三)厅堂式木构架
### (四)楼阁木构架
#### 1. 层叠构架楼阁
#### 2. 混合整体构架楼阁
## 二、材、栔、分°模数
## 三、定"地盘"——木构架的平面设定
## 四、定"侧样"——木构架的剖面设定
### (一)下份
### (二)中份
#### 1. 柱高
#### 2. 铺作高
### (三)上份
#### 1. 架深
#### 2. 檐出
#### 3. 举折
## 五、再谈定"侧样"中的架深
## 六、定"正样"——木构架的立面设定
### (一)影响房屋立面的八种因素
### (二)关于间广和柱高的讨论
### (三)屋顶端部处理
#### 1. 四阿殿脊槫增出
#### 2. 不厦两头(两下、两厦)造屋顶的出际
#### 3. 厦两头造与九脊殿
### (四)屋顶转角做法
#### 1. 角梁后尾支撑结构
#### 2. 角椽布置
### (五)制图学上的创新——"变角立面图"
## 七、大木构件
### (一)柱
### (二)梁
### (三)额
### (四)角梁
### (五)槫及橑檐枋
### (六)椽
### (七)蜀柱、叉手、托脚
### (八)串、襻间
### (九)替木
# 第二章注释
# 第三章 铺 作
## 一、概说
### (一)出跳承檐
### (二)计心造、偷心造、影栱
### (三)补间铺作的布置
#### 1. 朵数
#### 2. 间距
#### 3. 合理性
## 二、厅堂铺作
### (一)单斗只替
### (二)把头绞项作
### (三)斗口跳
### (四)四铺作
### (五)五铺作
### (六)六铺作
### (七)厅堂室内斗栱
####

In [25]:
import os

def save_output(text, folder_path="./output", file_name="cleaned_data.md"):
    """
    将文本保存到指定文件夹和文件名
    :param text: 要保存的内容
    :param folder_path: 文件夹路径 (例如 'C:/Documents' 或 './results')
    :param file_name: 文件名 (例如 'final.md')
    """
    # 1. 合并路径
    full_path = os.path.join(folder_path, file_name)
    
    try:
        # 2. 检查并创建文件夹
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"📂 已创建新目录: {folder_path}")

        # 3. 执行写入
        with open(full_path, "w", encoding="utf-8") as f:
            f.write(text)
        
        print(f"✅ 保存成功！")
        print(f"📍 完整路径: {os.path.abspath(full_path)}")
        
    except Exception as e:
        print(f"❌ 写入失败，错误原因: {e}")

# --- 运行设置 ---
MY_FOLDER = r"knowledgeBase\cleaned_data"  # 在此处修改文件夹路径
MY_FILE = "TOC_cleaned.md"   # 在此处修改文件名

save_output(step2_text, folder_path=MY_FOLDER, file_name=MY_FILE)

✅ 保存成功！
📍 完整路径: c:\Users\junqiang\myProjects\myArAppRag\knowledgeBase\cleaned_data\TOC_cleaned.md


In [ ]:
from dotenv import load_dotenv
load_dotenv()  # 自动寻找 .env 文件并加载到 os.environ

True

In [24]:
import os
from openai import OpenAI

def llm_semantic_clean(dirty_text):
    prompt = f"""
    Task: 你是一个专业的传统建筑古籍文献数字化专家，专门负责对《营造法式》相关目录数据进行深度语义清洗。
    Cleanup Requirements:
    1. 清除冗余符号： 彻底删除残留的引导点（·）、虚线、表格符（|）、以及行尾分散的数字（页码）。
    2. 处理 OCR 乱码： 识别并删除无语义的噪声字符，例如：N, □, ≠, .1∓, I, l 等。
    3. 语义纠错（核心）： 根据《营造法式》的专业背景，修正 OCR 识别错字。 例如将“营造法式”误识为“营造法试”，“木构架”误识为“木拱架”等。
    4. 规范层级： 确保 Markdown 标题层级（# 到 ####）逻辑连续。如果发现某一行虽然符合正则但语义上明显是上一行的延续，请合并。
    Output Format:
    1. Cleaned Markdown: 请直接给出清洗还原后干净、准确的 Markdown 文本。
    2. Changelog Table: 请列出一个表格，对比说明你修改了哪些关键错误及其原因。表格列名为：[原始文本] | [修改后文本] | [错误类型/原因]。
    待清洗内容：
    {dirty_text}
    """
    client = OpenAI(
    api_key = os.getenv("SEUAI_API_KEY"),
    base_url = "http://10.128.202.100:3010/v1"
)
    response = client.chat.completions.create(
        model="qwen3-235b-a22b-thinking-2507",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1  # 保持输出稳定，不发散
    )
    return response.choices[0].message.content

# 示例调用
cleaned_result = llm_semantic_clean(step2_text)
print(cleaned_result)

### Cleaned Markdown
# 目 录  
自 序  
说 明  
修订说明  
# 第一章 总 论  
## 一、《营造法式》的性质与特点  
### (一)"营造法式"是一种建筑工程预算定额  
### (二)李诫《营造法式》的编写体例  
### (三)编写工作紧密结合实际  
## 二、《营造法式》与江南建筑的关系  
## 三、《营造法式》的内容取舍  
## 四、宋代官式建筑分类  
## 五、研究方法的讨论  
# 第一章注释  
# 第二章 木构架  
## 一、宋代官式建筑木构架的基本类型  
### (一)柱梁作  
### (二)殿阁式木构架  
#### 1. 柱框层  
#### 2. 铺作层  
#### 3. 屋盖层  
### (三)厅堂式木构架  
### (四)楼阁木构架  
#### 1. 层叠构架楼阁  
#### 2. 混合整体构架楼阁  
## 二、材、栔、分模数  
## 三、定"地盘"——木构架的平面设定  
## 四、定"侧样"——木构架的剖面设定  
### (一)下份  
### (二)中份  
#### 1. 柱高  
#### 2. 铺作高  
### (三)上份  
#### 1. 架深  
#### 2. 檐出  
#### 3. 举折  
## 五、再谈定"侧样"中的架深  
## 六、定"正样"——木构架的立面设定  
### (一)影响房屋立面的八种因素  
### (二)关于间广和柱高的讨论  
### (三)屋顶端部处理  
#### 1. 四阿殿脊槫增出  
#### 2. 不厦两头(两下、两厦)造屋顶的出际  
#### 3. 厦两头造与九脊殿  
### (四)屋顶转角做法  
#### 1. 角梁后尾支撑结构  
#### 2. 角椽布置  
### (五)制图学上的创新——"变角立面图"  
## 七、大木构件  
### (一)柱  
### (二)梁  
### (三)额  
### (四)角梁  
### (五)槫及橑檐枋  
### (六)椽  
### (七)蜀柱、叉手、托脚  
### (八)串、襻间  
### (九)替木  
# 第二章注释  
# 第三章 铺 作  
## 一、概说  
### (一)出跳承檐  
### (二)计心造、偷心造、影栱  


In [1]:
import fitz  # PyMuPDF
import os

def export_pdf_bookmarks(pdf_path):
    # 1. 检查文件是否存在
    if not os.path.exists(pdf_path):
        print(f"错误：找不到文件 '{pdf_path}'，请检查路径。")
        return

    try:
        # 2. 打开 PDF
        doc = fitz.open(pdf_path)
        toc = doc.get_toc()  # 获取原始书签数据列表
        
        if not toc:
            print("提示：该 PDF 文件内部没有书签数据（逻辑目录为空）。")
            return

        # 3. 准备输出文件名
        output_filename = os.path.splitext(pdf_path)[0] + "_目录.txt"
        
        print(f"\n正在提取: {pdf_path}")
        print("-" * 60)
        
        with open(output_filename, "w", encoding="utf-8") as f:
            # 写入标题行
            header = f"{'层级':<5} | {'页码':<5} | {'章节标题'}"
            f.write(header + "\n" + "="*60 + "\n")
            print(header)
            print("-" * 60)

            for entry in toc:
                level, title, page = entry[0], entry[1], entry[2]
                
                # 构建缩进和视觉引导符
                indent = "    " * (level - 1)
                visual_prefix = "|-- " if level > 1 else ""
                
                # 格式化单行内容
                display_line = f"L{level:<4} | P{page:<4} | {indent}{visual_prefix}{title}"
                file_line = f"[{level}] 页码:{page:<4} | {indent}{visual_prefix}{title}"
                
                # 逻辑 A：打印到控制台
                print(display_line)
                
                # 逻辑 B：输出到本地文件
                f.write(file_line + "\n")

        print("-" * 60)
        print(f"✅ 导出成功！本地文件已生成：\n👉 {os.path.abspath(output_filename)}")

    except Exception as e:
        print(f"运行出错：{e}")
    finally:
        if 'doc' in locals():
            doc.close()

# --- 执行部分 ---
# 替换为你电脑上的 PDF 绝对路径或相对路径
pdf_file = r"knowledgeBase\pdfParsed_input\《营造法式》解读 第2版 (潘谷西；何建中著) (Z-Library).pdf" 
export_pdf_bookmarks(pdf_file)


正在提取: knowledgeBase\pdfParsed_input\《营造法式》解读 第2版 (潘谷西；何建中著) (Z-Library).pdf
------------------------------------------------------------
层级    | 页码    | 章节标题
------------------------------------------------------------
L1    | P1    | 封面
L1    | P3    | 书名
L1    | P4    | 版权
L1    | P5    | 前言
L1    | P8    | 目录
L1    | P16   | 第一章  总论
L2    | P16   |     |-- 一、《营造法式》的性质与特点
L3    | P16   |         |-- （一）“营造法式”是一种建筑工程预算定额
L3    | P17   |         |-- （二）李诫《营造法式》的编写体例
L3    | P19   |         |-- （三）编写工作紧密结合实际
L2    | P20   |     |-- 二、《营造法式》与江南建筑的关系
L2    | P29   |     |-- 三、《营造法式》的内容取舍
L2    | P31   |     |-- 四、宋代官式建筑分类
L2    | P32   |     |-- 五、研究方法的讨论
L2    | P34   |     |-- 第一章注释
L1    | P36   | 第二章  木构架
L2    | P36   |     |-- 一、宋代官式建筑木构架的基本类型
L3    | P36   |         |-- （一）柱梁作
L3    | P38   |         |-- （二）殿阁式木构架
L4    | P38   |             |-- 1.柱框层
L4    | P42   |             |-- 2.铺作层
L4    | P43   |             |-- 3.屋盖层
L3    | P44   |         |-- （三）厅堂式木构架
L3    | P56   |  

2. 删除页面分割线、引用、脚注等无用文本与符号

3. 错误识别字型识别与还原

4. markdown图片语法还原